<a href="https://colab.research.google.com/github/omkarpat/EmpatheticDialoguesEmotionDetection/blob/master/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 245\ Project

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 245 Project


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 9.1MB/s 
     |████████████████████████████████| 3.0MB 44.1MB/s 
     |████████████████████████████████| 1.1MB 40.3MB/s 
     |████████████████████████████████| 890kB 58.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=30c09fb349f383c4355f439e82fd64e378ac497eca475133f0585868ecefa8de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!ls

 basic_collapsed_emotion_model2_save
 basic_collapsed_emotion_model4_save
 collapsing_categories.txt
 Data
'Emotion Detection.gslides'
 emotion_model1_save
 emotion_model2_save
 emotion_model3_save
 emotion_model4_save
'Final PPT.gslides'
 less_restrictive_collapsed_emotion_model2_save
 less_restrictive_collapsed_emotion_model4_save
 model4_more_restrictive_collapsed_emotion_save
 more_restrictive_collapsed_emotion_model2_save
 more_restrictive_collapsed_emotion_model4_save
 pos_neg_model1_save
 pos_neg_model2_save
 pos_neg_model3_save
 pos_neg_model4_save
 valence_model1_save
 valence_model2_save
 valence_model3_save
 valence_model4_save


In [4]:
!ls Data

'Data Exploration.ipynb'
 data_fixed_train_complement_analysis.csv
 data_fixed_train.json
 data_sample_100.json
 data_sample_10_complement_analysis.csv
 data_sample_10.json
 data_sample_10_processed.json
 data_sample_fixed_processed_final.json
 data_sample_fixed_processed_model1_final.csv
 data_sample_fixed_processed_model2_final.csv
 data_sample_fixed_processed_model3_final.csv
 data_sample_fixed_processed_model4_final.csv
 fixed
 fixed_test.csv
 fixed_test.json
 fixed_train_516.csv
 fixed_valid.csv
 fixed_valid.json
'informative words.ipynb'
 model1_predictions_test.csv
 model1_predictions_train.csv
 model1_predictions_valid.csv
 model2_predictions_test.csv
 model2_predictions_train.csv
 model2_predictions_valid.csv
 model3_predictions_test.csv
 model3_predictions_train.csv
 model3_predictions_valid.csv
 model4_predictions_test.csv
 model4_predictions_train.csv
 model4_predictions_valid.csv
'Model Metrics.gsheet'
 Raw
 test_fixed_processed_final.json
 test_fixed_processed_model1_fina

In [5]:
dataset_name = "model4"
model_name = "model4"
model_type = "more_restrictive_collapsed_emotion" #pos_neg | emotion | valence | basic_collapsed_emotion | more_restrictive_collapsed_emotion | less_restrictive_collapsed_emotion

In [6]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [7]:
import pandas as pd
import numpy as np
train_filename = "Data/data_sample_fixed_processed_" + dataset_name + "_final.csv"
valid_filename = "Data/valid_fixed_processed_" + dataset_name + "_final.csv"
test_filename = "Data/test_fixed_processed_" + dataset_name + "_final.csv"

train_df = pd.read_csv(train_filename, sep='|', error_bad_lines=False)
train_df = train_df.dropna()
valid_df = pd.read_csv(valid_filename, sep='|', error_bad_lines=False)
valid_df = valid_df.dropna()
test_df = pd.read_csv(test_filename, sep='|', error_bad_lines=False)
test_df = test_df.dropna()

In [8]:
category2index_more_restrictive_model2 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "sentimental":3,
    "sad":4,
    "hopeful":5,
    "joyful":6,
    "impressed":7,
    "proud":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "trusting":13,
    "nostalgic":14,
    "excited":15,
    "disgusted":16,
    "caring":17,
    "prepared":18,
    "disappointed":19,
    "faithful":20,
    "confident":21,
    "apprehensive":22,
    "jealous":23,
    "embarrassed":24,
    "devastated":25,
    "annoyed":26,
    "content":27,
    "anticipating":28
}

category2index_more_restrictive_model4 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "nostalgic":3,
    "sentimental":3,
    "sad":4,
    "hopeful":5,
    "joyful":6,
    "impressed":7,
    "proud":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "trusting":13,
    "excited":14,
    "disgusted":15,
    "caring":16,
    "prepared":17,
    "disappointed":18,
    "faithful":19,
    "confident":20,
    "apprehensive":21,
    "jealous":22,
    "embarrassed":23,
    "devastated":24,
    "annoyed":25,
    "content":26,
    "anticipating":27
}

category2index_less_restrictive_model2 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "nostalgic":3,
    "sentimental":3,
    "anxious":4,
    "apprehensive":4,
    "excited":5,
    "anticipating":5,
    "sad":6,
    "hopeful":7,
    "joyful":8,
    "impressed":9,
    "proud":10,
    "grateful":11,
    "lonely":12,
    "surprised":13,
    "trusting":14,
    "disgusted":15,
    "caring":16,
    "prepared":17,
    "disappointed":18,
    "faithful":19,
    "confident":20,
    "jealous":21,
    "embarrassed":22,
    "devastated":23,
    "annoyed":24,
    "content":25    
}

category2index_less_restrictive_model4 = {
    "guilty":0,
    "ashamed":0,
    "furious":1,
    "angry":1,
    "terrified":2,
    "afraid":2,
    "nostalgic":3,
    "sentimental":3,
    "anxious":4,
    "apprehensive":4,
    "excited":5,
    "joyful":5,
    "anticipating":6,
    "sad":7,
    "hopeful":8,
    "impressed":9,
    "proud":10,
    "grateful":11,
    "lonely":12,
    "surprised":13,
    "trusting":14,
    "disgusted":15,
    "caring":16,
    "prepared":17,
    "disappointed":18,
    "faithful":19,
    "confident":20,
    "jealous":21,
    "embarrassed":22,
    "devastated":23,
    "annoyed":24,
    "content":25    
}

category2index_basic_collapsed_emotion = {
    "nostalgic":1,
    "sentimental":1,
    "sad":2,
    "devastated":2,
    "hopeful":3,
    "joyful":4,
    "impressed":5,
    "terrified":6,
    "afraid":6,
    "proud":7,
    "grateful":8,
    "lonely":9,
    "surprised":10,
    "anxious":11,
    "guilty":12,
    "trusting":13,
    "excited":15,
    "disgusted":16,
    "caring":17,
    "furious":18,
    "angry":18,
    "prepared":19,
    "disappointed":20,
    "faithful":21,
    "confident":22,
    "apprehensive":23,
    "jealous":24,
    "embarrassed":25,
    "annoyed":26,
    "ashamed":27,
    "content":28,
    "anticipating":29
}

category2index = {
    "invalid":0,
    "sentimental":1,
    "sad":2,
    "hopeful":3,
    "joyful":4,
    "impressed":5,
    "terrified":6,
    "proud":7,
    "afraid":8,
    "grateful":9,
    "lonely":10,
    "surprised":11,
    "anxious":12,
    "guilty":13,
    "trusting":14,
    "nostalgic":15,
    "excited":16,
    "disgusted":17,
    "caring":18,
    "furious":19,
    "angry":20,
    "prepared":21,
    "disappointed":22,
    "faithful":23,
    "confident":24,
    "apprehensive":25,
    "jealous":26,
    "embarrassed":27,
    "devastated":28,
    "annoyed":29,
    "ashamed":30,
    "content":31,
    "anticipating":32
}

if model_name == "model2":
  if model_type == "more_restrictive_collapsed_emotion":
    category2index = category2index_more_restrictive_model2
  elif model_type == "less_restrictive_collapsed_emotion":
    category2index = category2index_less_restrictive_model2
if model_name == "model4":
  if model_type == "more_restrictive_collapsed_emotion":
    category2index = category2index_more_restrictive_model4
  elif model_type == "less_restrictive_collapsed_emotion":
    category2index = category2index_less_restrictive_model4
if model_type == "basic_collapsed_emotion":
  category2index = category2index_basic_collapsed_emotion

number_of_labels = len(set(category2index.values()))
#print(category2index)
print("Preparing data for training %s, with %s. Total Classes: %s" % (model_name, model_type, number_of_labels))

index2category = {}
for key, value in category2index.items():
  index2category[value] = key

def category2label(category):
  for key in category2index.keys():
    if key == category:
      return category2index[key]
  return 0

def label2emotion(labels):
  emotion_list = []
  for label in labels:
    emotion_list.append(index2category[label])
  return emotion_list

def label2category(labels):
  emotion_list = []
  for label in labels:
    if label == 0:
      emotion_list.append("negative")
    else:
      emotion_list.append("positive")
  return emotion_list

Preparing data for training model4, with more_restrictive_collapsed_emotion. Total Classes: 28


In [10]:
print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
print('Number of valid sentences: {:,}\n'.format(valid_df.shape[0]))
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

if "emotion" in model_type:
  train_df['label'] = train_df['emotion'].apply(lambda x: int(category2label(x.lower())))
  valid_df['label'] = valid_df['emotion'].apply(lambda x: int(category2label(x.lower())))
  test_df['label'] = test_df['emotion'].apply(lambda x: int(category2label(x.lower())))
else:
  train_df['label'] = train_df['processed_emotion'].apply(lambda x: 0 if x == "negative" else 1)
  valid_df['label'] = valid_df['processed_emotion'].apply(lambda x: 0 if x == "negative" else 1)
  test_df['label'] = test_df['processed_emotion'].apply(lambda x: 0 if x == "negative" else 1)

# Display 10 random rows from the data.
train_df.sample(10)

Number of training sentences: 19,533

Number of valid sentences: 2,758

Number of test sentences: 2,538



,emotion,processed_emotion,text,vader_neg,vader_neu,vader_pos,vader_compound,textblob,flair_value,flair_confidence,label
4150,joyful,positive,I could ask for a better wife! My wife is my s...,0.000,0.699,0.301,0.9467,0.321364,POSITIVE,0.999958,6
3273,anticipating,positive,I am really excited for the new super smash br...,0.026,0.619,0.355,0.9764,0.115960,POSITIVE,0.999962,27
4719,proud,positive,Managed to get a huge bonus for hard work. So ...,0.034,0.745,0.221,0.9601,0.143434,NEGATIVE,0.649291,8
17263,caring,positive,I felt like sharing a story about last week fo...,0.017,0.727,0.256,0.9751,0.375000,POSITIVE,0.938712,16
8513,surprised,positive,I was at a local restaurant that my wife reall...,0.000,0.730,0.270,0.9790,0.212821,POSITIVE,0.999456,11
7264,surprised,positive,I got a letter in the mail. Apparently a dista...,0.078,0.676,0.246,0.9345,0.121429,POSITIVE,0.987066,11
2548,sentimental,positive,I love watching the movies I watched as a kid ...,0.000,0.584,0.416,0.9744,0.441667,POSITIVE,0.999461,3
4993,excited,positive,I signed up for a cruise! I haven't been on on...,0.000,0.773,0.227,0.9588,0.493125,POSITIVE,0.996583,14
11491,angry,negative,I am so pissed of at President Trump I am so p...,0.249,0.639,0.112,-0.8360,0.175000,POSITIVE,0.943997,1
5150,anticipating,positive,I'm going on a date this weekend. I'm pretty a...,0.076,0.640,0.284,0.9313,0.166667,POSITIVE,0.999694,27


In [11]:
def compute_category(value):
  if value == 0:
    return "neutral"
  elif float(value) < 0:
    return "negative"
  else:
    return "positive"

In [ ]:
train_df['flair_value'] = train_df['flair_value'].apply(lambda x: x.lower())
valid_df['flair_value'] = valid_df['flair_value'].apply(lambda x: x.lower())
test_df['flair_value'] = test_df['flair_value'].apply(lambda x: x.lower())
train_df['textblob_category'] = train_df['textblob'].apply(lambda x: compute_category(x))
valid_df['textblob_category'] = valid_df['textblob'].apply(lambda x: compute_category(x))
test_df['textblob_category'] = test_df['textblob'].apply(lambda x: compute_category(x))
train_df['vader_category'] = train_df['vader_compound'].apply(lambda x: compute_category(x))
valid_df['vader_category'] = valid_df['vader_compound'].apply(lambda x: compute_category(x))
test_df['vader_category'] = test_df['vader_compound'].apply(lambda x: compute_category(x))
train_df

,emotion,processed_emotion,text,vader_neg,vader_neu,vader_pos,vader_compound,textblob,flair_value,flair_confidence,textblob_category,vader_category
0,sentimental,positive,I remember going to the fireworks with my best...,0.075,0.600,0.325,0.9803,0.450000,positive,0.995377,positive,positive
1,sad,negative,One year during christmas_comma_ i did not get...,0.193,0.752,0.054,-0.8626,-0.134286,positive,0.574868,negative,negative
2,hopeful,positive,I have been in college for four years and am o...,0.000,0.937,0.063,0.3612,0.000000,positive,0.988017,neutral,positive
3,joyful,positive,You never know how happy you can be until you ...,0.195,0.633,0.172,-0.0096,0.480000,positive,0.935849,positive,negative
4,impressed,positive,the redsox have a chance to break the amount o...,0.054,0.702,0.244,0.8588,-0.093750,positive,0.998813,negative,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
19528,afraid,negative,living out in the country when it gets dark it...,0.056,0.944,0.000,-0.3369,-0.147222,positive,0.891851,negative,negative
19529,trusting,positive,I took my boots off after a long day of workin...,0.059,0.941,0.000,-0.6083,-0.016667,negative,0.999897,negative,negative
19530,nostalgic,positive,Anytime I see a mall_comma_ I get nostalgic ab...,0.101,0.710,0.189,0.7263,-0.215000,positive,0.994397,negative,positive
19531,anticipating,positive,i cant wait to play the new madden_comma_ ive ...,0.050,0.737,0.213,0.8251,0.354545,positive,0.998630,positive,positive


In [ ]:
actual_labels_train = train_df.processed_emotion.values
actual_labels_valid = valid_df.processed_emotion.values
actual_labels_test = test_df.processed_emotion.values
vader_labels_train = train_df.vader_category.values
vader_labels_valid = valid_df.vader_category.values
vader_labels_test = test_df.vader_category.values
textblob_labels_train = train_df.textblob_category.values
textblob_labels_valid = valid_df.textblob_category.values
textblob_labels_test = test_df.textblob_category.values
flair_labels_train = train_df.flair_value.values
flair_labels_valid = valid_df.flair_value.values
flair_labels_test = test_df.flair_value.values

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print("Classwise Report for Vader")
print("Training set")
print(classification_report(actual_labels_train, vader_labels_train))
print("Validation set")
print(classification_report(actual_labels_valid, vader_labels_valid))
print("Testing set")
print(classification_report(actual_labels_test, vader_labels_test))

print("Classwise Report for Textblob")
print("Training set")
print(classification_report(actual_labels_train, textblob_labels_train))
print("Validation set")
print(classification_report(actual_labels_valid, textblob_labels_valid))
print("Testing set")
print(classification_report(actual_labels_test, textblob_labels_test))

print("Classwise Report for Flair")
print("Training set")
print(classification_report(actual_labels_train, flair_labels_train))
print("Validation set")
print(classification_report(actual_labels_valid, flair_labels_valid))
print("Testing set")
print(classification_report(actual_labels_test, flair_labels_test))

Classwise Report for Vader
Training set


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    negative       0.85      0.64      0.73      9673
     neutral       0.00      0.00      0.00         0
    positive       0.73      0.84      0.78      9860

    accuracy                           0.74     19533
   macro avg       0.52      0.49      0.50     19533
weighted avg       0.79      0.74      0.75     19533

Validation set
              precision    recall  f1-score   support

    negative       0.83      0.61      0.70      1374
    positive       0.68      0.87      0.76      1299

    accuracy                           0.74      2673
   macro avg       0.75      0.74      0.73      2673
weighted avg       0.76      0.74      0.73      2673

Testing set
              precision    recall  f1-score   support

    negative       0.84      0.62      0.71      1225
    positive       0.71      0.89      0.79      1313

    accuracy                           0.76      2538
   macro avg       0.78      0.75      0.75    

In [12]:
sentences_train = train_df.text.values
labels_train = train_df.label.values
sentences_valid = valid_df.text.values
labels_valid = valid_df.label.values
sentences_test = test_df.text.values
labels_test = test_df.label.values

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

longer_list = []
for index, sent in enumerate(sentences_train):
  if len(tokenizer.encode(sent, add_special_tokens=True)) > 500:
    longer_list.append(index)
sentences_train = np.delete(sentences_train, longer_list)
labels_train = np.delete(labels_train, longer_list)

max_len = 0
# For every sentence...
for sent in sentences_train:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max train sentence length: ', max_len)

max_len = 0
# For every sentence...
for sent in sentences_valid:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max valid sentence length: ', max_len)

max_len = 0
# For every sentence...
for sent in sentences_test:
    try:
      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent, add_special_tokens=True)
    except:
      print(sent)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max test sentence length: ', max_len)

def tokenize(sentences, labels):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.

      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          truncation = 'only_first',
                          max_length = 512,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.from_numpy(labels)

  return(input_ids, attention_masks, labels)

input_ids_train, attention_masks_train, labels_train = tokenize(sentences_train, labels_train)
input_ids_valid, attention_masks_valid, labels_valid = tokenize(sentences_valid, labels_valid)
input_ids_test, attention_masks_test, labels_test = tokenize(sentences_test, labels_test)

Loading BERT tokenizer...


Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_valid = TensorDataset(input_ids_valid, attention_masks_valid, labels_valid)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

#Use the given training and validation datasets
train_dataset, val_dataset = dataset_train, dataset_valid

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))
print('{:>5,} test samples'.format(len(dataset_test)))

19,533 training samples
2,763 validation samples
2,542 test samples


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
output_dir = "./" + model_type + "_" + model_name + "_save/"

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

print(model_type + "_" + model_name + " loaded...")

pos_neg_model1 loaded...


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

def predict(dataset):

  # Create the DataLoader for  dataset.
  prediction_data = dataset
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


  # Prediction on test set

  print('Predicting labels for {:,} sentences...'.format(len(dataset)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []

  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  print('DONE.')
  return predictions, true_labels

test_predictions, test_true_labels = predict(dataset_test)
valid_predictions, valid_true_labels = predict(dataset_valid)
train_predictions, train_true_labels = predict(dataset_train)

Predicting labels for 2,542 sentences...
DONE.
Predicting labels for 2,763 sentences...
DONE.
Predicting labels for 19,533 sentences...
DONE.


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def metrics(predictions, true_labels):
  for i in range(len(predictions)):
    if i == 0:
      pred_flat = np.argmax(predictions[i], axis=1).flatten()
      confidence_flat = np.amax(predictions[i], axis=1).flatten()
      labels_flat = true_labels[i].flatten()
    else:
      pred_flat = np.append(pred_flat, np.argmax(predictions[i], axis=1).flatten())
      confidence_flat = np.append(confidence_flat, np.amax(predictions[i], axis=1).flatten())
      labels_flat = np.append(labels_flat, true_labels[i].flatten())

  if model_type == "emotion":
    pred_flat = label2emotion(pred_flat)
    labels_flat = label2emotion(labels_flat)
  else:
    pred_flat = label2category(pred_flat)
    labels_flat = label2category(labels_flat)
  print("Classwise Report")
  print(classification_report(labels_flat, pred_flat))
  return pred_flat, confidence_flat, labels_flat
print("Metrics on testing set")
pred_flat_test, confidence_flat_test, labels_flat_test = metrics(test_predictions, test_true_labels)
print("Metrics on validation set")
pred_flat_valid, confidence_flat_valid, labels_flat_valid = metrics(valid_predictions, valid_true_labels)
print("Metrics on training set")
pred_flat_train, confidence_flat_train, labels_flat_train = metrics(train_predictions, train_true_labels)

Metrics on testing set
Classwise Report
              precision    recall  f1-score   support

    negative       0.90      0.91      0.90      1227
    positive       0.91      0.90      0.91      1315

    accuracy                           0.91      2542
   macro avg       0.91      0.91      0.91      2542
weighted avg       0.91      0.91      0.91      2542

Metrics on validation set
Classwise Report
              precision    recall  f1-score   support

    negative       0.90      0.90      0.90      1377
    positive       0.90      0.90      0.90      1386

    accuracy                           0.90      2763
   macro avg       0.90      0.90      0.90      2763
weighted avg       0.90      0.90      0.90      2763

Metrics on training set
Classwise Report
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00      9673
    positive       1.00      1.00      1.00      9860

    accuracy                           1.00     19533
   m

In [ ]:
if model_type == "emotion":
  final_train_df['emotion_actual'] = pd.DataFrame(labels_flat_train, columns=['emotion_actual'])
  final_train_df['emotion_pred'] = pd.DataFrame(pred_flat_train, columns=['emotion_pred'])
  final_valid_df['emotion_actual'] = pd.DataFrame(labels_flat_valid, columns=['emotion_actual'])
  final_valid_df['emotion_pred'] = pd.DataFrame(pred_flat_valid, columns=['emotion_pred'])
  final_test_df['emotion_actual'] = pd.DataFrame(labels_flat_test, columns=['emotion_actual'])
  final_test_df['emotion_pred'] = pd.DataFrame(pred_flat_test, columns=['emotion_pred'])
else:
  final_train_df = pd.DataFrame(sentences_train, columns=['sentences'])
  final_train_df['pos_neg_actual'] = pd.DataFrame(labels_flat_train, columns=['pos_neg_actual'])
  final_train_df['pos_neg_pred'] = pd.DataFrame(pred_flat_train, columns=['pos_neg_pred'])
  final_valid_df = pd.DataFrame(sentences_valid, columns=['sentences'])
  final_valid_df['pos_neg_actual'] = pd.DataFrame(labels_flat_valid, columns=['pos_neg_actual'])
  final_valid_df['pos_neg_pred'] = pd.DataFrame(pred_flat_valid, columns=['pos_neg_pred'])
  final_test_df = pd.DataFrame(sentences_test, columns=['sentences'])
  final_test_df['pos_neg_actual'] = pd.DataFrame(labels_flat_test, columns=['pos_neg_actual'])
  final_test_df['pos_neg_pred'] = pd.DataFrame(pred_flat_test, columns=['pos_neg_pred'])
final_train_df

,sentences,pos_neg_actual,pos_neg_pred,emotion_actual,emotion_pred
0,I remember going to the fireworks with my best...,positive,positive,sentimental,sentimental
1,One year during christmas_comma_ i did not get...,negative,negative,sad,sad
2,I have been in college for four years and am o...,positive,positive,hopeful,hopeful
3,You never know how happy you can be until you ...,positive,positive,joyful,joyful
4,the redsox have a chance to break the amount o...,positive,positive,impressed,impressed
...,...,...,...,...,...
19525,living out in the country when it gets dark it...,negative,negative,afraid,afraid
19526,I took my boots off after a long day of workin...,positive,positive,trusting,embarrassed
19527,Anytime I see a mall_comma_ I get nostalgic ab...,positive,positive,nostalgic,nostalgic
19528,i cant wait to play the new madden_comma_ ive ...,positive,positive,anticipating,anticipating


In [ ]:
final_train_df.to_csv("Data/" + model_name + "_" + "predictions_train.csv", index=False)
final_valid_df.to_csv("Data/" + model_name + "_" + "predictions_valid.csv", index=False)
final_test_df.to_csv("Data/" + model_name + "_" + "predictions_test.csv", index=False)